In [13]:
import sys
import os
import numpy as np
from encode import Chromagram
from encode import Encoder
from data_loader import data_loader
import pickle

import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [14]:
def get_note_without_octave(note):
    return ''.join([char for char in note if not char.isdigit()])

In [15]:
#directory of chord wav files
data_dir = "/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords"

#a list of all the wav files in the directory
wav_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.wav')]

#this function uses the filename to get the chord type, chord name, and root note name
def parse_filename(filename):
    base_name = os.path.basename(filename)
    root_name = os.path.splitext(base_name)[0]
    parts = root_name.split('_')
    chord_type = parts[1] 
    chord_name = parts[2]
    midi_note_name = parts[3]
    return chord_type, chord_name, midi_note_name

#this is chord type, chord name, and root note name for each file in wav_files
labels = [parse_filename(fname) for fname in wav_files]

In [19]:
labels

[('minorseventh', 'G#3', '56'),
 ('minorseventh', 'C3', '48'),
 ('minorseventh', 'A3', '57'),
 ('minorseventh', 'A4', '69'),
 ('minorseventh', 'G#3', '56'),
 ('minorseventh', 'A#4', '70'),
 ('minorseventh', 'D6', '86'),
 ('minorseventh', 'D#4', '63'),
 ('minorseventh', 'C1', '24'),
 ('minorseventh', 'A1', '33'),
 ('minorseventh', 'E4', '64'),
 ('minorseventh', 'E6', '88'),
 ('minorseventh', 'F1', '29'),
 ('minorseventh', 'E3', '52'),
 ('minorseventh', 'C6', '84'),
 ('minorseventh', 'F#4', '66'),
 ('minorseventh', 'G#4', '68'),
 ('minorseventh', 'C#4', '61'),
 ('minorseventh', 'F2', '41'),
 ('minorseventh', 'D#3', '51'),
 ('minorseventh', 'D#5', '75'),
 ('minorseventh', 'F6', '89'),
 ('minorseventh', 'F#3', '54'),
 ('minorseventh', 'A#3', '58'),
 ('minorseventh', 'G#1', '32'),
 ('minorseventh', 'C5', '72'),
 ('minorseventh', 'F1', '29'),
 ('minorseventh', 'F2', '41'),
 ('minorseventh', 'E5', '76'),
 ('minorseventh', 'F#6', '90'),
 ('minorseventh', 'A5', '81'),
 ('minorseventh', 'G1', '3

In [21]:
#this is the directory where the spike data will be saved
def create_numerical_labels(labels):
    # Extract chord name from labels
    chord_name = [chord_name for _, chord_name, _ in labels]
    roots = [get_note_without_octave(note) for note in chord_name]

    #extract chord type from labels
    chord_types = [chord_type for chord_type, _, _ in labels]
    
    # Get unique root notes and assign a unique number to each
    unique_chord_names = sorted(list(set(chord_name)))
    unique_roots = sorted(list(set(roots)))
    unique_types = sorted(list(set(chord_types)))
    
    chord_to_num = {root: i for i, root in enumerate(unique_chord_names)}
    root_to_num = {note: i for i, note in enumerate(unique_roots)}
    type_to_num = {chord_type: i for i, chord_type in enumerate(unique_types)}

    num_to_chord = {i: root for root, i in chord_to_num.items()}
    num_to_root = {i: note for note, i in root_to_num.items()}
    num_to_type = {i: chord_type for chord_type, i in type_to_num.items()}

    # Convert root notes to numerical labels
    numerical_label_chord_name = [chord_to_num[chord_name] for chord_name in chord_name]
    numerical_label_roots = [root_to_num[root] for root in roots]
    numerical_label_types = [type_to_num[chord_type] for chord_type in chord_types]
     
    return chord_name, roots, chord_to_num, root_to_num, numerical_label_chord_name, numerical_label_roots, num_to_chord, num_to_root, numerical_label_types, num_to_type

# Create numerical labels for root notes
chord_name, roots, chord_to_num, root_to_num, numerical_label_chord_name, numerical_label_roots, num_to_chord, num_to_root, numerical_label_types, num_to_type = create_numerical_labels(labels)

In [24]:
# print("chord_name: ", chord_name)
# print("roots: ", roots)
# print("chord_to_num: ", chord_to_num)
# print("root_to_num: ", root_to_num)
# print("numerical_label_chord_name: ", numerical_label_chord_name)
# print("numerical_label_roots: ", numerical_label_roots)
# print("num_to_chord: ", num_to_chord)
print("num_to_root: ", num_to_root)
# print("numerical_label_types: ", numerical_label_types)
print("num_to_type: ", num_to_type)



num_to_root:  {0: 'A', 1: 'A#', 2: 'B', 3: 'C', 4: 'C#', 5: 'D', 6: 'D#', 7: 'E', 8: 'F', 9: 'F#', 10: 'G', 11: 'G#'}
num_to_type:  {0: 'augmentedtriad', 1: 'diminishedtriad', 2: 'dom7ninth', 3: 'dom7seventh', 4: 'majorseventh', 5: 'majortriad', 6: 'minorseventh', 7: 'minortriad', 8: 'sus2triad', 9: 'sus4triad'}


AttributeError: type object 'data_loader' has no attribute 'chroma_files'

In [ ]:
#this is the directory where the spike data will be saved
spike_data_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords"

#this for loop goes through each wav file in all_files, creates a chromagram, and saves the chromagram as a numpy array
for wav_path in wav_files:
    chroma = Chromagram(wav_path)
    chroma_array = chroma.chromagram
    
    # Replicate the directory structure for spike data
    relative_path = os.path.relpath(wav_path, data_dir)
    spike_filename = os.path.join(spike_data_dir, relative_path.replace('.wav', '.npy'))
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(spike_filename), exist_ok=True)
    
    # Save the spike data to disk using pickle
    np.save(spike_filename, chroma_array)

#this is a list of all the spike data files, just like all_
chroma_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.npy')]

# Chroma files are not needed to make y (aka the labels for training).
The chroma making loop is only really needed once. we can call chroma files from the chroma folder. So lets make two classes one for creating/loading chroma files and and one for creating/loadind the labels.

In [ ]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(chroma_files, numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

In [ ]:
y_test_encoded.shape

In [ ]:
class labeler_loader:
    def __init__(self):
        #directory of chord wav files
        self.source_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords"
        #a list of all the wav files in the directory
        self.wav_files = self._get_wav_files()
        #chord type, chord name, and root note name
        self.labels = self._get_labels()
        self.numerical_label_chord, self.numerical_label_roots = self.create_numerical_labels(self.labels)
        
    def _get_wav_files(self):
        return [os.path.join(root, fname)
                for root, dirs, files in os.walk(self.source_dir)
                for fname in files if fname.endswith('.wav')]

    #this function uses the filename to get the chord type, chord name, and root note name
    def _parse_filename(self, filename):
        base_name = os.path.basename(filename)
        root_name = os.path.splitext(base_name)[0]
        parts = root_name.split('_')
        chord_type = parts[1] 
        chord_name = parts[2]
        midi_note_name = parts[3]
        return chord_type, chord_name, midi_note_name

    #this is chord type, chord name, and root note name for each file in wav_files
    def _get_labels(self):
        return [self._parse_filename(fname) for fname in self.wav_files]
    
    def _get_note_without_octave(self, note):
        return ''.join([char for char in note if not char.isdigit()])
    
    
    #this is the directory where the spike data will be saved
    def create_numerical_labels(self, labels):
        # Extract chord name from labels
        chord_names = [chord_name for _, chord_name, _ in self.labels]
        roots = [self._get_note_without_octave(note) for note in chord_names]
        
        # Get unique root notes and assign a unique number to each
        unique_chord_names = sorted(list(set(chord_names)))
        unique_roots = sorted(list(set(roots)))
        
        chord_to_num = {root: i for i, root in enumerate(unique_chord_names)}
        root_to_num = {note: i for i, note in enumerate(unique_roots)}

        # Convert root notes to numerical labels
        numerical_label_chord = [chord_to_num[chord] for chord in chord_names]
        numerical_label_roots = [root_to_num[root] for root in roots]
        
        return numerical_label_chord, numerical_label_roots
    

In [ ]:
class chroma_loader:
    def __init__(self):
        #this is the directory where the spike data will be saved
        self.destination_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords"
        self.chroma_files = self._get_chroma_files()

    #this for loop goes through each wav file in all_files, creates a chromagram, and saves the chromagram as a numpy array
    def create_npy_chroma(self):
        for wav_path in labeler_loader().wav_files:
            chroma = Chromagram(wav_path)
            chroma_array = chroma.chromagram
            
            # Replicate the directory structure for spike data
            relative_path = os.path.relpath(wav_path, labeler_loader().source_dir)
            spike_filename = os.path.join(self.destination_dir, relative_path.replace('.wav', '.npy'))
            
            # Ensure the directory exists
            os.makedirs(os.path.dirname(spike_filename), exist_ok=True)
            
            # Save the spike data to disk using pickle
            np.save(spike_filename, chroma_array)
            
    def _get_chroma_files(self):
        return [os.path.join(root, fname)
                for root, dirs, files in os.walk(labeler_loader().source_dir)
                for fname in files if fname.endswith('.npy')]


In [ ]:
class data_loader:
    def __init__(self):
        #directories used
        self.source_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords"
        self.destination_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords"
        #lists of files
        self.wav_files = self._get_files(self.source_dir, '.wav')
        self.chroma_files = self._get_files(self.destination_dir, '.npy') #We use this in training
        #labels and numerical labels
        self.labels = self._get_labels()      
        self.y, self.root_to_num = self.create_numerical_labels(self.labels) #We use this in training
        
    def _get_files(self, directory, extension):
        return [os.path.join(root, fname)
                for root, dirs, files in os.walk(directory)
                for fname in files if fname.endswith(extension)]

    def _parse_filename(self, filename):
        base_name = os.path.basename(filename)
        root_name = os.path.splitext(base_name)[0]
        parts = root_name.split('_')
        chord_type = parts[1] 
        chord_name = parts[2]
        midi_note_name = parts[3]
        return chord_type, chord_name, midi_note_name
    #this is chord type, chord name, and root note name for each file in wav_files
    def _get_labels(self):
        return [self._parse_filename(fname) for fname in self.wav_files]
    
    def _get_note_without_octave(self, note):
        return ''.join([char for char in note if not char.isdigit()])
    
    #this is the directory where the spike data will be saved
    def create_numerical_labels(self, labels):
        # Extract chord name from labels
        chord_names = [chord_name for _, chord_name, _ in self.labels]
        roots = [self._get_note_without_octave(note) for note in chord_names]
        
        # Get unique root notes and assign a unique number to each
        unique_chord_names = sorted(list(set(chord_names)))
        unique_roots = sorted(list(set(roots)))
        
        chord_to_num = {root: i for i, root in enumerate(unique_chord_names)}
        root_to_num = {note: i for i, note in enumerate(unique_roots)}

        # Convert root notes to numerical labels
        numerical_label_chord = [chord_to_num[chord] for chord in chord_names]
        numerical_label_roots = [root_to_num[root] for root in roots]
        
        return numerical_label_chord, numerical_label_roots               


# ~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config)

net = neuro.Network()
net.set_properties(proc.get_network_properties())

In [ ]:
n_inputs = 12
n_hidden = 100
n_outputs = 88
n_total = n_inputs + n_hidden + n_outputs

moa = neuro.MOA()
moa.seed(42)

In [ ]:
for i in range(n_inputs):
    node = net.add_node(i)
    net.add_input(i)
    net.randomize_node_properties(moa,node)

for i in range(n_outputs):
    node = net.add_node(n_inputs+i)
    net.add_output(i)
    net.randomize_node_properties(moa,node)

In [ ]:
eo_params = {
    "population_size": 100,
    "starting_nodes": n_total,
    "starting_edges": n_total,
    "multi_edges": 0.05,
    "merge_rate": .01,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "add_node_rate": 0.5,
    "add_edge_rate": 0.9,
    "delete_node_rate": 0.4,
    "delete_edge_rate": 0.8,
    "node_params_rate": 1.5,
    "edge_params_rate": 1.5,
    "net_params_rate": .01,
    "num_mutations": 5,
    "random_factor": 0.05,
    "num_best": 3,

    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
}


In [ ]:
ex1 ='/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_ninth/NordGrand1_dom7ninth_G3_55.npy'


In [ ]:
evolver = eons.EONS(eo_params)

In [ ]:
ex1_spike = Encoder(ex1).spikes

In [ ]:
proc.load_network(net)

In [ ]:
proc.apply_spikes(ex1_spike)

In [ ]:
# TODO: see what the output of proc is

In [ ]:
encoder_params = {
    "dmin": [-1],
    "dmax": [1.5],
    "encoders": [{ "rate": { "subinterval_size": 1 } }],
    "interval": 8.8,
    }

In [ ]:
encoder = neuro.EncoderArray(encoder_params)